In [ ]:
# import functions from the shadow_model folder
import sys
sys.path.append('..')
from shadow_model.ray_utils import *
from shadow_model.visual_utils import *
from shadow_model.voxel_utils import *
import argparse
import os

In [ ]:
import laspy
import numpy as np

# Parameters
params = ModelParams(
    base_k3=0.4,
    base_k4=0.5,
    base_k5=0.6,
    building_k=8.0,
    vegetation_weight=1.0,  # Adjust for your data scale
    density_weight=0.01,  # Adjust for your data scale
    shadow_gamma=1
)

# Load LiDAR data
las_path = "../data/houselas_re_veg_2.las"
las_file = laspy.read(las_path)
utm_coords = np.vstack((las_file.x, las_file.y, las_file.z)).T
classifications = las_file.classification

# Parameters
voxel_size = 1
target_coords = np.array([532886, 6983516])
target_z = find_z_at_target(utm_coords, target_coords)
radius = 50
altitude_range = np.arange(0, 91, 1)
azimuth_range = np.arange(0, 361, 1)

# Voxelization
voxel_grid, min_bounds = voxelize(utm_coords, classifications, voxel_size)
voxel_grid = fill_building_voxel_columns(voxel_grid)
voxel_grid = expand_target_area(voxel_grid, min_bounds, voxel_size, target_coords, target_z, target_voxel_radius=1)

# Compute Shadow Matrix
shadow_matrix = create_shadow_matrix(altitude_range, azimuth_range, voxel_grid, min_bounds, voxel_size, target_coords, target_z, radius, params)

# Visualize Shadow
plot_shadow_polar(shadow_matrix, altitude_range, azimuth_range)

# Save shadow results
df_shadow = create_dataframe(shadow_matrix, altitude_range, azimuth_range)
df_shadow.to_csv("../results/shadow_matrix_1.csv")